In [18]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras.applications import VGG19

In [19]:
# Define the number of classes
num_classes = 4  # glioma, meningioma, notumer, pituitary

In [20]:
# Load the VGG19 base model (excluding the top layers)
base_model = VGG19(weights='imagenet', include_top=False)

# Add custom top layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [21]:
# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

In [22]:
# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

In [23]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0   

In [24]:
train_folder = r"D:\Dataset\Brain tumer\Training"

test_folder = r"D:\Dataset\Brain tumer\Testing"

In [27]:

# Train the model using the training data
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_folder,
        target_size=(224, 224),
        batch_size=16,
        class_mode='categorical')

# Validate the model using the testing data
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
        test_folder,
        target_size=(224, 224),
        batch_size=16,
        class_mode='categorical')

# Train the model


Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [28]:
model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // 32,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // 32,
        epochs=50)

# Save the model



Epoch 1/50
178/178 [==============================] - 38s 184ms/step - loss: 0.9535 - accuracy: 0.5569 - val_loss: 0.7060 - val_accuracy: 0.7078
Epoch 2/50
178/178 [==============================] - 32s 182ms/step - loss: 0.6082 - accuracy: 0.7237 - val_loss: 0.6187 - val_accuracy: 0.7281
Epoch 3/50
178/178 [==============================] - 32s 181ms/step - loss: 0.5222 - accuracy: 0.7669 - val_loss: 0.5719 - val_accuracy: 0.7922
Epoch 4/50
178/178 [==============================] - 32s 180ms/step - loss: 0.4417 - accuracy: 0.8181 - val_loss: 0.5746 - val_accuracy: 0.7766
Epoch 5/50
178/178 [==============================] - 32s 180ms/step - loss: 0.4346 - accuracy: 0.8336 - val_loss: 0.6755 - val_accuracy: 0.7094
Epoch 6/50
178/178 [==============================] - 32s 179ms/step - loss: 0.4285 - accuracy: 0.8385 - val_loss: 0.5171 - val_accuracy: 0.8031
Epoch 7/50
178/178 [==============================] - 32s 178ms/step - loss: 0.3336 - accuracy: 0.8768 - val_loss: 0.4757 - val_ac

In [29]:
model.save('Brain_Tumer.h5')